In [1]:
import requests
import pandas as pd
from time import sleep
import os
import json

In [ ]:
endpoint = 'https://free-nba.p.rapidapi.com/'
headers = {
	"X-RapidAPI-Key": "563b99e32emsh860d6965169a64dp1da03bjsnf21eaf721a45",
	"X-RapidAPI-Host": "free-nba.p.rapidapi.com"
}

players_endpoint = os.path.join(endpoint, 'games')

querystring = {'page': 0, 'per_page': 50}

responses = []
max_pages = float('inf')
i = 0
num_timeouts = 0
success = 0
while i < max_pages:
  querystring['page'] = i
  response = requests.get(players_endpoint, headers=headers, params=querystring)
  while response.status_code == 429:
    num_timeouts += 1
    print(f'Uh oh! We got put in time out >.< {num_timeouts}th time')
    sleep(10)
    response = requests.get(players_endpoint, headers=headers, params=querystring)
  response = json.loads(response.text)

  if max_pages == float('inf'):
    max_pages = response['meta']['total_pages']
  i += 1
  responses.extend(response['data'])
  success += 1

print('successfully collected data')

Uh oh! We got put in time out >.< 1th time
Uh oh! We got put in time out >.< 2th time
Uh oh! We got put in time out >.< 3th time
Uh oh! We got put in time out >.< 4th time
Uh oh! We got put in time out >.< 5th time
Uh oh! We got put in time out >.< 6th time
Uh oh! We got put in time out >.< 7th time
Uh oh! We got put in time out >.< 8th time
Uh oh! We got put in time out >.< 9th time
Uh oh! We got put in time out >.< 10th time
Uh oh! We got put in time out >.< 11th time
Uh oh! We got put in time out >.< 12th time
Uh oh! We got put in time out >.< 13th time
Uh oh! We got put in time out >.< 14th time
Uh oh! We got put in time out >.< 15th time
Uh oh! We got put in time out >.< 16th time
Uh oh! We got put in time out >.< 17th time


In [120]:
df = pd.DataFrame(responses)
df

,id,date,home_team,home_team_score,period,postseason,season,status,time,visitor_team,visitor_team_score
0,47179,2019-01-30T00:00:00.000Z,"{'id': 2, 'abbreviation': 'BOS', 'city': 'Bost...",126,4,False,2018,Final,,"{'id': 4, 'abbreviation': 'CHA', 'city': 'Char...",94
1,48751,2019-02-09T00:00:00.000Z,"{'id': 2, 'abbreviation': 'BOS', 'city': 'Bost...",112,4,False,2018,Final,,"{'id': 13, 'abbreviation': 'LAC', 'city': 'LA'...",123
2,48739,2019-02-08T00:00:00.000Z,"{'id': 23, 'abbreviation': 'PHI', 'city': 'Phi...",117,4,False,2018,Final,,"{'id': 8, 'abbreviation': 'DEN', 'city': 'Denv...",110
3,48740,2019-02-08T00:00:00.000Z,"{'id': 30, 'abbreviation': 'WAS', 'city': 'Was...",119,4,False,2018,Final,,"{'id': 6, 'abbreviation': 'CLE', 'city': 'Clev...",106
4,48746,2019-02-08T00:00:00.000Z,"{'id': 26, 'abbreviation': 'SAC', 'city': 'Sac...",102,4,False,2018,Final,,"{'id': 16, 'abbreviation': 'MIA', 'city': 'Mia...",96
...,...,...,...,...,...,...,...,...,...,...,...
29245,30726,2012-11-18T00:00:00.000Z,"{'id': 9, 'abbreviation': 'DET', 'city': 'Detr...",103,4,False,2012,Final,,"{'id': 2, 'abbreviation': 'BOS', 'city': 'Bost...",83
29246,30727,2012-11-19T00:00:00.000Z,"{'id': 30, 'abbreviation': 'WAS', 'city': 'Was...",89,4,False,2012,Final,,"{'id': 12, 'abbreviation': 'IND', 'city': 'Ind...",96
29247,30728,2012-11-21T00:00:00.000Z,"{'id': 16, 'abbreviation': 'MIA', 'city': 'Mia...",113,4,False,2012,Final,,"{'id': 17, 'abbreviation': 'MIL', 'city': 'Mil...",106
29248,30729,2012-11-23T00:00:00.000Z,"{'id': 8, 'abbreviation': 'DEN', 'city': 'Denv...",102,4,False,2012,Final,,"{'id': 10, 'abbreviation': 'GSW', 'city': 'Gol...",91


In [121]:
home_team_series = pd.DataFrame(df['home_team'].to_list())

In [122]:
home_team_series.columns = 'home_team_' + home_team_series.columns

In [123]:
df1 = pd.concat([df, home_team_series], axis=1)

In [124]:
df1 = df1.drop('home_team', axis=1)

In [132]:
columns = df1.columns.tolist()
new_columns = columns[:2] + columns[10:] + columns[2:10]
df2 = df1[new_columns]

In [134]:
visitor_team_series = pd.DataFrame(df['visitor_team'].to_list())
visitor_team_series.columns = 'visitor_team_' + visitor_team_series.columns
df3 = pd.concat([df2, visitor_team_series], axis = 1)
df3 = df3.drop('visitor_team', axis = 1)

In [136]:
df3

,id,date,home_team_id,home_team_abbreviation,home_team_city,home_team_conference,home_team_division,home_team_full_name,home_team_name,home_team_score,...,status,time,visitor_team_score,visitor_team_id,visitor_team_abbreviation,visitor_team_city,visitor_team_conference,visitor_team_division,visitor_team_full_name,visitor_team_name
0,47179,2019-01-30T00:00:00.000Z,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics,126,...,Final,,94,4,CHA,Charlotte,East,Southeast,Charlotte Hornets,Hornets
1,48751,2019-02-09T00:00:00.000Z,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics,112,...,Final,,123,13,LAC,LA,West,Pacific,LA Clippers,Clippers
2,48739,2019-02-08T00:00:00.000Z,23,PHI,Philadelphia,East,Atlantic,Philadelphia 76ers,76ers,117,...,Final,,110,8,DEN,Denver,West,Northwest,Denver Nuggets,Nuggets
3,48740,2019-02-08T00:00:00.000Z,30,WAS,Washington,East,Southeast,Washington Wizards,Wizards,119,...,Final,,106,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers
4,48746,2019-02-08T00:00:00.000Z,26,SAC,Sacramento,West,Pacific,Sacramento Kings,Kings,102,...,Final,,96,16,MIA,Miami,East,Southeast,Miami Heat,Heat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29245,30726,2012-11-18T00:00:00.000Z,9,DET,Detroit,East,Central,Detroit Pistons,Pistons,103,...,Final,,83,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics
29246,30727,2012-11-19T00:00:00.000Z,30,WAS,Washington,East,Southeast,Washington Wizards,Wizards,89,...,Final,,96,12,IND,Indiana,East,Central,Indiana Pacers,Pacers
29247,30728,2012-11-21T00:00:00.000Z,16,MIA,Miami,East,Southeast,Miami Heat,Heat,113,...,Final,,106,17,MIL,Milwaukee,East,Central,Milwaukee Bucks,Bucks
29248,30729,2012-11-23T00:00:00.000Z,8,DEN,Denver,West,Northwest,Denver Nuggets,Nuggets,102,...,Final,,91,10,GSW,Golden State,West,Pacific,Golden State Warriors,Warriors
